In [166]:
import stanfordnlp
stanfordnlp.download('en')
nlp = stanfordnlp.Pipeline(processors = "tokenize,mwt,lemma,pos")
doc = nlp("Event Management is the process that monitors all events that occur through the IT infrastructure to allow for normal operation and also to detect and escalate exception conditions.")
doc.sentences[0].print_tokens()


Using the default treebank "en_ewt" for language "en".
Would you like to download the models for: en_ewt now? (Y/n)

Default download directory: C:\Users\User\stanfordnlp_resources
Hit enter to continue or type an alternate directory.

Download location: C:\Users\User\stanfordnlp_resources\en_ewt_models.zip

Download complete.  Models saved to: C:\Users\User\stanfordnlp_resources\en_ewt_models.zip
Extracting models file for: en_ewt
Cleaning up...Done.
Use device: cpu
---
Loading: tokenize
With settings: 
{'model_path': 'C:\\Users\\User\\stanfordnlp_resources\\en_ewt_models\\en_ewt_tokenizer.pt', 'lang': 'en', 'shorthand': 'en_ewt', 'mode': 'predict'}
---
Loading: lemma
With settings: 
{'model_path': 'C:\\Users\\User\\stanfordnlp_resources\\en_ewt_models\\en_ewt_lemmatizer.pt', 'lang': 'en', 'shorthand': 'en_ewt', 'mode': 'predict'}
Building an attentional Seq2Seq model...
Using a Bi-LSTM encoder
Using soft attention for LSTM.
Finetune all embeddings.
[Running seq2seq lemmatizer with ed

100%|██████████| 235M/235M [04:02<00:00, 873kB/s] 


In [182]:
print(*[f'text: {word.text+" "}\tlemma: {word.lemma}\tupos: {word.upos}\txpos: {word.xpos}' for sent in doc.sentences for word in sent.words], sep='\n')




text: Event         	lemma: event	upos: NOUN	xpos: NN
text: Management         	lemma: management	upos: NOUN	xpos: NN
text: is         	lemma: be	upos: AUX	xpos: VBZ
text: the         	lemma: the	upos: DET	xpos: DT
text: process         	lemma: process	upos: NOUN	xpos: NN
text: that         	lemma: that	upos: PRON	xpos: WDT
text: monitors         	lemma: monitor	upos: VERB	xpos: VBZ
text: all         	lemma: all	upos: DET	xpos: DT
text: events         	lemma: event	upos: NOUN	xpos: NNS
text: that         	lemma: that	upos: PRON	xpos: WDT
text: occur         	lemma: occur	upos: VERB	xpos: VBP
text: through         	lemma: through	upos: ADP	xpos: IN
text: the         	lemma: the	upos: DET	xpos: DT
text: IT         	lemma: it	upos: NOUN	xpos: NN
text: infrastructure         	lemma: infrastructure	upos: NOUN	xpos: NN
text: to         	lemma: to	upos: PART	xpos: TO
text: allow         	lemma: allow	upos: VERB	xpos: VB
text: for         	lemma: for	upos: ADP	xpos: IN
text: normal         	le

In [284]:
with open("ITIL Books/ITIL 3/Service operation glossary/definitions.txt") as file:
    glossary_definitions = file.read()

definitions = [i.lower().replace("\n", " ").split('::') for i in glossary_definitions.split('\n\n')]

concepts_from_definitions = [i[0] for i in definitions]
# print(concepts_from_definitions)

with open("ITIL Books/ITIL 3/Service operation chapter 4/Service operation chapter 4 - 4.txt") as file:
    text = file.read()

# print(text)

ontology = {}

for index in range(0, len(definitions)):
    ontology[definitions[index][0]] = [f"definition:: {definitions[index][1]}"]

# print(ontology)

import nltk
# Tokenize into sentences
sentences = nltk.sent_tokenize(text)

# Split sentences into words
sentences = [nltk.word_tokenize(i.lower().replace("\n", " ")) for i in sentences]

sentences_containing_concepts = {}
for sentence in sentences:
    # print("---")
    count = 0
    for concept in concepts_from_definitions:
        concept_list = concept.split()
        
        # print(concept_list)
        
        # count = count + 1
        # if count is 1:
        #     print(sentence)
        if concept_list[0] in sentence:
            add_to_sentences_containing_concepts = True
            
            if len(concept_list) > 1:
                current_word_index = sentence.index(concept_list[0])
                concept_word_length_count = 1
                for concept_word in concept_list[1::]:
                    current_word_index = current_word_index + 1
                    
                    if sentence[current_word_index] != concept_word:
                        add_to_sentences_containing_concepts = False
                        break
                                            
                    # print(concept)
                    # print(concept_word)            
                
            if add_to_sentences_containing_concepts is True:
                sentence_string = " ".join(sentence)
                if sentence_string not in sentences_containing_concepts.keys():
                    sentences_containing_concepts[sentence_string] = []
                sentences_containing_concepts[sentence_string].append(concept)
            

# sentences_containing_concepts = [i for i in sentences if i in concepts_from_definitions]
 
# for sentence in sentences_containing_concepts:
#     print(sentence)

# print(sentences_containing_concepts)

# print(ontology)

for sentence, concepts in sentences_containing_concepts.items():
    for concept in concepts:
        ontology[concept].append(sentence)

# TODO: In definitions replace (book name) with own in book relation


In [287]:
import re
import random

question_template = "question_template(Question_number=1,Answer_number=4)::Question->What is the definition for {concept.name}?::Answer->{concept.definition}"

# print(question_template)

def get_definition(concept_name):
    return [i.split("::")[1] for i in ontology[concept_name] if i.split("::")[0] == "definition"][0]

class Concept:
    def __init__(self, name, ontology):
        self.name = name
        self.ontology = ontology
        self.attributes = ontology[self.name]
        # self.definition = [i.split("::")[1] for i in ontology[self.name] if i.split("::")[0] == "definition"][0]
        self.definition = get_definition(self.name)
        # self.book = re.findall("^\w*\(.*?\)", self.definition)
        # self.definition = re.sub("^\w*\(.*?\)", "", self.definition)         
        
    # def name(self):
    #     return self.name
    # 
    # def definition(self):
    #     return self.definition
    # 
    # def attributes(self):
    #     return self.attributes
    

class Question:
    def __init__(self, question_string, answer_list, question_params):
        self.question_string = question_string
        self.answer_list = answer_list
        self.parameters = question_params
        
    def print_question_and_answer(self):
        print(self.question_string)
        for answer in self.answer_list:
            print(f"- {answer}")
        
    def format_output(self):
        self.print_question_and_answer()
        
def get_incorrect_answers(correct_concept, number_of_incorrect_answers):
    incorrect_answers = []
    
    while(len(incorrect_answers) < int(number_of_incorrect_answers) - 1):
        incorrect_concept_name = list(ontology.keys())[random.randint(0, len(ontology))]
        if incorrect_concept_name != correct_concept.name:
            incorrect_answers.append(get_definition(incorrect_concept_name))
    
    return incorrect_answers
    
def replace_template_variables(template_format_string, concept):
    replaced_string = template_format_string

    replaced_string = re.sub("{.*?concept\.name.*?}", concept.name, template_format_string)
    replaced_string = re.sub("{.*?concept\.definition.*?}", concept.definition, replaced_string)\
    # replaced_string = re.sub("{.*?concept\.relation\.attribute*?}", concept.attrubute, replaced_string)
    
    return replaced_string

def parse_template_question(question_template_to_parse, concept_name=""):
    template_list = question_template_to_parse.split("::")
    question_parameters = template_list[0].split("(")[1].replace(")", "").split(",")    
    question_parameters = {item.split("=")[0]: item.split("=")[1] for item in question_parameters}
    question = template_list[1].split("->")[1]
    answer = template_list[2].split("->")[1]
        
    if concept_name == "":
        concept_name = list(ontology.keys())[random.randint(0, len(ontology))]
        
    # print(concept_name)
    
    concept = Concept(concept_name, ontology)
    
    # print(concept.definition)
    # print(concept.book)
    
    # print(concept.name)
    # print(concept.definition)
    # print(concept.attributes)
    
    # question = re.sub("{.*?concept\.name.*?}", concept.name, question)
    # question = re.sub("{.*?concept\.definition.*?}", concept.definition, question)
    # # question = re.sub("{.*?concept\.relation\.attribute*?}", concept.attrubute, question)
    # 
    # answer = re.sub("{.*?concept\.name.*?}", concept_name, answer)
    # answer = re.sub("{.*?concept\.definition.*?}", concept.definition, answer)
    # # answer = re.sub("{.*?concept\.relation\.definition*?}", ontology[concept_name], answer)
    
    # print(question)
    # print(answer)
    
    question = replace_template_variables(question, concept)
    answer = replace_template_variables(answer, concept)
    
    answer_list = [answer]
            
    if "Answer_number" in question_parameters.keys():
        answer_list = answer_list + get_incorrect_answers(concept, question_parameters["Answer_number"])
    else:
        answer_list = answer_list + get_incorrect_answers(concept, 4)
                
    return Question(question, answer_list, question_parameters)

def parse_template(template_to_parse, concept_name=""):
    if "question_template" in template_to_parse.split("::")[0]:
        return parse_template_question(template_to_parse)
    else:
        print(f"Error parsing template {template_to_parse}")
        return None
 
parsed_template = parse_template(question_template)

parsed_template.format_output()

# for sentence in sentences:
#     print(nltk.word_tokenize(sentence))
    
# print(nltk.tokenize.TextTilingTokenizer().tokenize(glossary_definitions))
# print(nltk.word_tokenize(glossary_definitions))

What is the definition for release record ?
-  (service transition) a record in the cmdb that defines the content of a release. a release record has relationships with all configuration items that are affected by the release.
-  cost resulting from running the it services. often repeating payments. for example staff costs, hardware maintenance and electricity (also known as â€˜current expenditureâ€™ or â€˜revenue expenditureâ€™). see also capital expenditure.
-  the implementation and management of quality it services that meet the needs of the business. it service management are performed by it service providers through an appropriate mix of people, process and information technology. see also service management.
-  (service operation) the function responsible for monitoring and control of the it services and it infrastructure. see also operations bridge.
